# Install kaggle-environments

In [1]:
# 1. Enable Internet in the Kernel (Settings side pane)

# 2. Curl cache may need purged if v0.1.6 cannot be found (uncomment if needed). 
#!curl -X PURGE https://pypi.org/simple/kaggle-environments

# ConnectX environment was defined in v0.1.6
#!pip install 'kaggle-environments>=0.1.6'

# Create ConnectX Environment

In [2]:
# from kaggle_environments import evaluate, make, utils
# import numpy as np
# import time

# env = make("connectx", debug=True)
# env.render()

# Mi Codigo

In [3]:
from abc import abstractmethod

class IGame:

    @abstractmethod
    def is_game_over(board, inarow):
        pass

    @abstractmethod
    def change_turn(player):
        pass

    @abstractmethod
    def get_open_cols(board):
        pass

    @abstractmethod
    def make_move(board, col, player):
        pass

    @abstractmethod
    def get_turn(board):
        pass

    @abstractmethod
    def print_board(board):
        pass

In [4]:
import numpy as np
from functools import lru_cache

class Connectx():
    def __init__(self, inarow, rows, cols):
        self.cols = cols
        self.rows = rows
        self.inarow = inarow
        

    def change_turn(self, player): 
        if player == '1':
            return '2'
        else:
            return '1'


    def get_open_cols(self, board):
        # board is serialized
        return [i for i in range(self.cols) if board[i] == '0']

    def make_move(self, board, col, player):

        entire_col = [board[i*self.cols + col] for i in range(self.rows)]
        for i, cell in enumerate(reversed(entire_col)):
            if cell == '0':
                board = board[:(self.rows-i-1)*self.cols + col] + player + board[(self.rows-i-1)*self.cols+col+1:]
                break
        return board

    def get_turn(self, board):
        # assuming that '1' always starts playing
        ones = board.count('1')
        twos = board.count('2')
        return '1' if ones == twos else '2'

    @lru_cache(maxsize=2048)
    def is_game_over(self, board, inarow):
        '''
        return the winner or '0' if there is no winner yet
        '''
        board2d = self.deserialize_board(board)
        
        # rows
        for i in range(self.rows):
            previous = ''
            count = 1
            for j in range(self.cols):
                current = board[i*self.cols + j]
                if previous != '0' and previous == current:
                    count += 1
                else:
                    count = 1
                if count == inarow:
                    return current
                previous = current
        
        # columns
        for i in range(self.cols):
            previous = ''
            count = 1
            for j in range(self.rows):
                current = board[j*self.cols + i]
                if previous != '0' and previous == current:
                    count += 1
                else:
                    count = 1
                if count == inarow:
                    return current
                previous = current
        
        # positive diagonal
        for row in range(self.rows-(inarow-1)):
            for col in range(self.cols-(inarow-1)):
                window = list(board2d[range(row, row+inarow), range(col, col+inarow)])
                
                if window.count('1') == inarow:
                    return '1'
                elif window.count('2') == inarow:
                    return '2'

        # negative diagonal
        for row in range(inarow-1, self.rows):
            for col in range(self.cols-(inarow-1)):
                window = list(board2d[range(row, row-inarow, -1), range(col, col+inarow)])
                if window.count('1') == inarow:
                    return '1'
                elif window.count('2') == inarow:
                    return '2'
        return '0' if board.count('0') != 0 else 'draw'

    def print_board(self, board):
        for i in range(self.rows):
            print('|', end='')
            for j in range(self.cols):
                print(board[i*self.cols + j], end='|')
            print()
        print(board)
    
    def serialize_board(self, board):
        return ''.join([str(cell) for row in board for cell in row])

    def deserialize_board(self, board: str):
        return np.frombuffer(board.encode(), dtype=np.int8).reshape((self.rows, self.cols))
    
    def get_inputs(self, board, turn):
        board2d = self.deserialize_board(board)
        x1 = np.where(board2d == b'2', 1, 0)
        x2 = np.where(board2d == b'1', 1, 0)
        if turn == '2':
            x3 = np.ones((self.rows, self.cols))
        else:
            x3 = np.zeros((self.rows, self.cols))
        a = np.expand_dims(np.array([x1, x2, x3]), axis=0)
        return a

In [5]:
import time
from functools import lru_cache

class MCTS:

    def __init__(self, game: IGame = None, model=None):
        self.explored = set()
        self.nodes_parameters = {} # fen: (N, V) N--> times visited, V-->value
        # self.UCT = {} # fen: UPC (upper confidence tree)
        self.C = 3 # aprox sqrt(2)
        self.game = game
        self.probabilities = {}
        self.model = model

    def get_value(self, result: str, player):
        if result == '1':
            return 1 if player == '1' else -1

        elif result == '2':
            return -1 if player == '1' else 1 
        else:
            return 0
        
    @lru_cache(maxsize=2048)
    def search(self, s):
        result = self.game.is_game_over(s, self.game.inarow)
        if result != '0':
            v = self.get_value(result, self.game.get_turn(s)) 
            if s not in self.nodes_parameters:
                self.nodes_parameters[s] = np.array((1, v))
            else:
                self.nodes_parameters[s][0] += 1
            return -v, 1

        childs = self.game.get_open_cols(s)

        if s in self.explored:
        # choose which node is going to be expanded
            best_uct = float('-inf')
            best_child = None
            turn = self.game.get_turn(s)
            # best_w = 0
            n_p = self.nodes_parameters[s][0] # parent's n
            for a in childs:
                s_child = self.game.make_move(s, a, turn)
                n, w = self.nodes_parameters[s_child]
                p = self.probabilities[s_child]    
                child_uct = self.get_UCT(n, w, n_p, p)
                if child_uct > best_uct:
                    best_uct = child_uct
                    best_child = a
            s_aux = self.game.make_move(s, best_child, self.game.get_turn(s))
            sum_v, sum_n = self.search(s_aux)
            # propagate the results
            self.nodes_parameters[s_aux][0] += sum_n
            self.nodes_parameters[s_aux][1] += sum_v

        
        else:
            turn = self.game.get_turn(s)
            if len(self.explored) == 0:
                probs, v = self.model.predict(self.game.get_inputs(s, turn))
                self.add_probs(s, probs)
            self.explored.add(s)
            sum_v = 0
            sum_n = 0
            for a in childs:
                s_child = self.game.make_move(s, a, turn)
                if s_child not in self.nodes_parameters:
                    # v = self.simulate(s_child, turn, turn)
                    probs, v = self.model.predict(self.game.get_inputs(s_child, turn))
                    self.add_probs(s_child, probs)
                    # print(probs.shape, v.shape, v)
                    self.nodes_parameters[s_child] = np.array((1, v.squeeze()))
                    sum_v += v
                    sum_n += 1
                
            self.nodes_parameters[s][0] += sum_n
            self.nodes_parameters[s][1] += sum_v

        return -sum_v, sum_n
    
    def add_probs(self, state, probs):
        open_cols = self.game.get_open_cols(state)
        not_open_cols = [i for i in range(self.game.cols) if i not in open_cols]
        probs = probs.reshape((self.game.cols,))
        probs[not_open_cols] = 0
        for a, prob in enumerate(np.ravel(probs)):
            child_state = self.game.make_move(state, a, self.game.get_turn(state))
            self.probabilities[child_state] = prob

    

    def simulate(self, s, color_playing, turn):
        result = self.game.is_game_over(s, self.game.inarow)
        while result == '0': 
            move = np.random.choice(self.game.get_open_cols(s))
            s = self.game.make_move(s, move, turn)
            result = self.game.is_game_over(s, self.game.inarow)
            turn = self.game.change_turn(turn)
        return self.get_value(result, color_playing)


    def get_UCT(self, n, w, n_p, p):
        # return w/n + self.C * np.sqrt(n_p) / (1 + n)
        return w/n + self.C * p * np.sqrt(np.log(n_p) / (n + 1))
    
    def get_pi(self, state, tau=1):
        pi = []
        turn = self.game.get_turn(state)
        open_cols =  self.game.get_open_cols(state)
        for a in range(self.game.cols):
            if a not in open_cols:
                pi.append(0)
            else:
                s_child = self.game.make_move(state, a, turn)
                pi.append(self.nodes_parameters[s_child][0])
        pi = np.power(pi, tau) 
        pi_sum = np.sum(pi)
        return np.divide(pi, pi_sum)
    
    def iterate(self, s, n_iters=None, time_limit=None):
        self.nodes_parameters[s] = np.array((1, 0))
        if time_limit is not None:
            end_time = time.time() + time_limit
            while time.time() < end_time:
                v, n = self.search(s)
                self.nodes_parameters[s][0] += n
                self.nodes_parameters[s][1] += v
        else:            
            for _ in range(n_iters):
                v, n = self.search(s)
                self.nodes_parameters[s][0] += n
                self.nodes_parameters[s][1] += v

    def best_move(self, board, turn, n_iters=None, time_limit=None):
        self.iterate(board, n_iters=n_iters, time_limit=time_limit)
        max_n = 0
        best_move = 3 # por ejemplo 3
        for a in self.game.get_open_cols(board):
            c_aux = self.game.make_move(board, a, turn)
            if self.game.is_game_over(c_aux, self.game.inarow) != '0':
                return a
            current_n = self.nodes_parameters[c_aux][0]
            if current_n > max_n:
                max_n = current_n
                best_move = a
            # print(f'N: {self.nodes_parameters[c_aux][0]} , V: {self.nodes_parameters[c_aux][1]}')
            # self.game.print_board(c_aux)
        return best_move



# Neural Network

In [6]:
from tensorflow.keras.layers import Input, Conv2D, Dense, BatchNormalization, ReLU, Add, Flatten
from tensorflow.keras import Model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
import tensorflow as tf

physical_devices = tf.config.list_physical_devices('GPU')
# physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.set_visible_devices(physical_devices[0], 'GPU')
    # tf.config.experimental.set_memory_growth(physical_devices[0], True)

class NeuralNetwok():
    

    def __init__(self, rows, cols, l_rate):
        self.rows = rows
        self.cols = cols
        self.l_rate = l_rate
        self.N_BLOCKS = 2
        self.KERNEL_SIZE = (3, 3) # the stride will be 1 (the default used by keras)
        self.FILTERS = 128
        self.model = self.create_model(rows, cols, self.N_BLOCKS)
        
    
    def create_model(self, rows, cols, n_blocks):
        # input of the neural network
        # 3 channels: one board with the position of th X's,
        # other with the positions of the O's and the last, all 1 if X plays or all 0 if O plays
        input_layer = Input(shape=(3, rows, cols))
        
        # body (residual blocks)
        x = self.add_convolutional_layer(input_layer, self.FILTERS, self.KERNEL_SIZE)
        for _ in range(n_blocks):
            x = self.add_residual_block(x)
            
        # add policy head
        policy_head_output = self.add_policy_head(x)
        
        # add value head
        value_head_output = self.add_value_head(x)
        
        model = Model(inputs=input_layer, outputs=[policy_head_output, value_head_output])
        # model.compile(
        #     optimizer = Adam(self.l_rate),
        #     loss = ['categorical_crossentropy','mean_squared_error']
        # )
        return model
        
    def add_residual_block(self, x):
        # first convolutional layer
        y = self.add_convolutional_layer(x, self.FILTERS, self.KERNEL_SIZE)
        
        # second convolutional layer
        y = Conv2D(self.FILTERS, self.KERNEL_SIZE, data_format='channels_first', padding='same')(y)
        y = BatchNormalization()(y)
        y = Add()([x, y])
        y = ReLU()(y)
        return y
    
    def add_convolutional_layer(self, x, filters, kernel_size):
        y = Conv2D(filters, kernel_size, data_format='channels_first', padding='same')(x)
        # padding has to be 'same' in order to keep the dimensions right
        # data_format=channels_first corresponds to inputs with shape (batch_size, channels, height, width).
        y = BatchNormalization()(y)
        y = ReLU()(y)
        return y
    
    def add_policy_head(self, x):
        y = self.add_convolutional_layer(x, 2, (1, 1))
        y = Flatten()(y)
        y = Dense(self.cols, activation='softmax')(y) # TODO: podria llegar a hacerse de tamaño rows*cols, ni idea
        # move logit probabilities
        return y
    
    def add_value_head(self, x):
        y = self.add_convolutional_layer(x, 1, (1, 1))
        y = Flatten()(y)
        y = Dense(128)(y)
        y = ReLU()(y)
        y = Dense(1, activation='tanh')(y)
        return y
    
    def predict(self, x):
        return self.model.predict(x, verbose=0)
    
    def train(self, x, y):
        self.model.compile(
            optimizer = Adam(self.l_rate),
            loss = ['categorical_crossentropy','mean_squared_error']
        )
        with tf.device('/gpu:0'):
            history = self.model.fit(epochs=25,
                                 batch_size=32,
                                 x=x, 
                                 y=y,
                                 verbose=1)
        
            
    def save(self, file_name):
        self.model.save(file_name)

    @staticmethod
    def load_network(file_name, lr=0.01):
        model = load_model(file_name)
        nn = NeuralNetwok(6, 7, lr)
        nn.model = model
        return nn

2024-04-22 21:41:39.762190: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-22 21:41:39.789241: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-22 21:41:40.204507: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-04-22 21:41:40.649277: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-22 21:41:40.649437: 

In [11]:
from threading import Thread, Barrier

class Player:

    def __init__(self, mcts):
        self.mcts = mcts

    def play(self, iterations, num_games, starting_iteration=0, n_threads=4):
        for i in range(starting_iteration, iterations):
            neural_network = self.mcts.model
            neural_network.save(f'model_iter_{i}.h5')

            # initialize the arrays
            # x, pi_s, results = self.generate_examples() # TODO: usar el metodo paralelo
            examples = self.generate_examples_parallel(f'model_iter_{i}.h5', n_threads)
            x, pi_s, results = examples[0]
            for example in examples[1:]:
                    x = np.vstack((x, example[0]))
                    print(example[1].shape)
                    pi_s = np.vstack((pi_s, example[1]))
                    results = np.concatenate((results, example[2]))
            print(pi_s.shape)
            for j in range(num_games - 1):
                print(f'generating data for iteration {i} and game {j}')
                # states, pi, result = self.generate_examples()
                # x = np.vstack((x, states))
                # print(pi.shape)
                # pi_s = np.vstack((pi_s, pi))
                # results = np.concatenate((results, result))
                examples = self.generate_examples_parallel(f'model_iter_{i}.h5', n_threads)
                for example in examples:
                    x = np.vstack((x, example[0]))
                    print(example[1].shape)
                    pi_s = np.vstack((pi_s, example[1]))
                    results = np.concatenate((results, example[2]))

            y = (pi_s, results.reshape(-1, 1))
            print(x.shape)
            print(y[0].shape)
            print(y[1].shape)
            neural_network.train(x, y)

            old_network = NeuralNetwok.load_network(f'model_iter_{i}.h5')
            neural_network.save(f'model_iter_{i}_trained.h5')
            win_rate = self.simulate_games_parallel(f'model_iter_{i}.h5', f'model_iter_{i}_trained.h5', games_per_thread=10, n_threads=n_threads)
            # win_rate = self.simulate_games(old_network, 25) # TODO: usar variable global
            print(f'winrate of the {i}th iteration: {win_rate}')
            # if the trained network is not much better than the previous one, then the previous one remains as the current network
            if win_rate < 0.55:
                self.mcts.model = old_network


    def generate_examples(self):
        states = []
        pi_s = []
        board = '0' * (self.mcts.game.rows * self.mcts.game.cols)
        result = '0'
        current_turn = '1'
        while result == '0':
            move = self.mcts.best_move(board, current_turn, n_iters=25)
            pi = self.mcts.get_pi(board)
            print(f'pi: {pi}')
            pi_s.append(pi)
            board = self.mcts.game.make_move(board, move, current_turn)
            states.append(self.mcts.game.get_inputs(board, current_turn)[0])
            result = self.mcts.game.is_game_over(board, self.mcts.game.inarow)
            current_turn = self.mcts.game.change_turn(current_turn)
        total_moves = len(states)
        results = np.ones(total_moves)
        self.mcts.game.print_board(board)
        if result == '1':
            results[1::2] = -1
        elif result == '2':
            results[::2] = -1
        else:
            results = np.zeros(total_moves)

        return (states, np.array(pi_s), results)

    def generate_examples_parallel(self, path_current_model, n_threads):
        barrier = Barrier(n_threads + 1)
        examples = [0] * n_threads
        rows = self.mcts.game.rows
        cols = self.mcts.game.cols
        inarow = self.mcts.game.inarow
        def generate(thread_id):
            model = NeuralNetwok.load_network(path_current_model)
            game = Connectx(inarow, rows, cols)
            mcts = MCTS(game, model)
            states = []
            pi_s = []
            board = '0' * (rows * cols)
            result = '0'
            current_turn = '1'
            while result == '0':
                move = mcts.best_move(board, current_turn, n_iters=25)
                pi = mcts.get_pi(board)
                pi_s.append(pi)
                board = game.make_move(board, move, current_turn)
                states.append(game.get_inputs(board, current_turn)[0])
                result = game.is_game_over(board, game.inarow)
                current_turn = game.change_turn(current_turn)
            total_moves = len(states)
            results = np.ones(total_moves)
            if result == '1':
                results[1::2] = -1
            elif result == '2':
                results[::2] = -1
            else:
                results = np.zeros(total_moves)
            examples[thread_id] = (states, np.array(pi_s), results)
            print(f'thread {thread_id} ended, result: {result}')
            barrier.wait()

        for thread_id in range(n_threads):
            t = Thread(target=generate, args=[thread_id])
            t.start()
        barrier.wait()
        return examples

    def simulate_games(self, old_network, num_games):
        old_mcts = MCTS()
        old_mcts.game = self.mcts.game
        old_mcts.model = old_network
        num_wins = 0
        for i in range(num_games):
            color_old_network = '1' if np.random.uniform() < 0.5 else '2'
            color_new_network = '2' if color_old_network == '1' else '1'
            result = self.play_one_game(old_mcts, color_old_network)
            if result == color_new_network:
                num_wins += 1
        return num_wins / num_games

    def play_one_game(self, rival, rival_color):
        board = '0' * (self.mcts.game.rows * self.mcts.game.cols)
        result = '0'
        is_rival_turn = rival_color == '1'
        current_player_color = '2' if is_rival_turn else '1'
        while result == '0':
            if is_rival_turn:
                move = rival.best_move(board, rival_color, n_iters=33) # TODO: usar variables globales para n_iters
                board = self.mcts.game.make_move(board, move, rival_color)
            else:
                move = self.mcts.best_move(board, current_player_color, n_iters=33)
                board = self.mcts.game.make_move(board, move, current_player_color)
            is_rival_turn = not is_rival_turn
            result = self.mcts.game.is_game_over(board, self.mcts.game.inarow)
        return result

    def simulate_games_parallel(self, old_network_path, current_network_path, games_per_thread, n_threads):
        barrier = Barrier(n_threads + 1)
        rows = self.mcts.game.rows
        cols = self.mcts.game.cols
        inarow = self.mcts.game.inarow
        game = self.mcts.game
        wins = [0] * n_threads
        def play_game(thread_id):
            old_network = NeuralNetwok.load_network(old_network_path)
            current_network = NeuralNetwok.load_network(current_network_path)
            for i in range(games_per_thread):
                old_mcts = MCTS(game, old_network)
                current_mcts = MCTS(game, current_network)
                board = '0' * (rows * cols)
                result = '0'
                color_old_network = '1' if np.random.uniform() < 0.5 else '2'
                is_old_net_turn = color_old_network == '1'
                current_player_color = '2' if is_old_net_turn else '1'
                while result == '0':
                    if is_old_net_turn:
                        move = old_mcts.best_move(board, color_old_network, n_iters=33) # TODO: usar variables globales para n_iters
                        board = game.make_move(board, move, color_old_network)
                    else:
                        move = current_mcts.best_move(board, current_player_color, n_iters=33)
                        board = game.make_move(board, move, current_player_color)
                    is_old_net_turn = not is_old_net_turn
                    result = game.is_game_over(board, inarow)

                if result == current_player_color:
                    wins[thread_id] += 1
                game.print_board(board)
                print(f'result of game {i} in thread {thread_id}: {result}')
            barrier.wait()

        for thread_id in range(n_threads):
            t = Thread(target=play_game, args=[thread_id])
            t.start()
        barrier.wait()
        print(wins)
        return sum(wins) / (games_per_thread * n_threads)

# Train the neural network

In [12]:
nn = NeuralNetwok(6, 7, 0.01)
#plot_model(nn.model, show_shapes=True)

In [13]:
game = Connectx(4, 6, 7)
nn = NeuralNetwok(6, 7, 0.01)
mcts = MCTS(game, nn)
player = Player(mcts)

In [14]:
player.play(100, 5, n_threads=4)

/tmp/ipykernel_1618635/2772718001.py:79: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  self.nodes_parameters[s][1] += sum_v
/tmp/ipykernel_1618635/2772718001.py:134: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  self.nodes_parameters[s][1] += v


thread 3 ended, result: 1
thread 0 ended, result: 1
thread 2 ended, result: 1
thread 1 ended, result: 1
(15, 7)
(15, 7)
(15, 7)
(60, 7)
generating data for iteration 0 and game 0


thread 1 ended, result: 1
thread 2 ended, result: 1
thread 0 ended, result: 1
thread 3 ended, result: 1
(15, 7)
(15, 7)
(15, 7)
(15, 7)
generating data for iteration 0 and game 1


thread 1 ended, result: 1
thread 2 ended, result: 1
thread 3 ended, result: 1
thread 0 ended, result: 1
(15, 7)
(15, 7)
(15, 7)
(15, 7)
generating data for iteration 0 and game 2


thread 2 ended, result: 1
thread 0 ended, result: 1
thread 1 ended, result: 1
thread 3 ended, result: 1
(15, 7)
(15, 7)
(15, 7)
(15, 7)
generating data for iteration 0 and game 3


thread 0 ended, result: 1
thread 2 ended, result: 1
thread 1 ended, result: 1
thread 3 ended, result: 1
(15, 7)
(15, 7)
(15, 7)
(15, 7)
(300, 3, 6, 7)
(300, 7)
(300, 1)
Epoch 1/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 33ms/step - loss: 3.0286
Epoch 2/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 2.4004
Epoch 3/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 2.2893
Epoch 4/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 2.2291
Epoch 5/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 2.1215
Epoch 6/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 2.1010
Epoch 7/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 2.0723
Epoch 8/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 2.0387
Epoch 9/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 1.9993
Epoch 10/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 1.9999
Epoch 11/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 1.9870
Epoch 12/25
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 1.9704
Epoch 13/25
10/10 ━━━━━━━━━━━━━━━━━━

|2|2|0|0|0|0|0|
|1|1|0|0|0|0|0|
|2|2|0|0|0|0|0|
|1|1|0|0|0|0|0|
|2|2|2|0|0|0|0|
|1|1|1|1|0|0|0|
220000011000002200000110000022200001111000
result of game 0 in thread 0: 1
|2|2|0|0|0|0|0|
|1|1|0|0|0|0|0|
|2|2|0|0|0|0|0|
|1|1|0|0|0|0|0|
|2|2|2|0|0|0|0|
|1|1|1|1|0|0|0|
220000011000002200000110000022200001111000
result of game 0 in thread 3: 1
|2|2|0|0|0|0|0|
|1|1|0|0|0|0|0|
|2|2|0|0|0|0|0|
|1|1|0|0|0|0|0|
|2|2|2|0|0|0|0|
|1|1|1|1|0|0|0|
220000011000002200000110000022200001111000
result of game 0 in thread 1: 1
|2|2|0|0|0|0|0|
|1|1|0|0|0|0|0|
|2|2|0|0|0|0|0|
|1|1|0|0|0|0|0|
|2|2|2|0|0|0|0|
|1|1|1|1|0|0|0|
220000011000002200000110000022200001111000
result of game 0 in thread 2: 1
|2|2|0|0|0|0|0|
|1|1|0|0|0|0|0|
|2|2|0|0|0|0|0|
|1|1|0|0|0|0|0|
|2|2|2|0|0|0|0|
|1|1|1|1|0|0|0|
220000011000002200000110000022200001111000
result of game 1 in thread 0: 1
|2|2|0|0|0|0|0|
|1|1|0|0|0|0|0|
|2|2|0|0|0|0|0|
|1|1|0|0|0|0|0|
|2|2|2|0|0|0|0|
|1|1|1|1|0|0|0|
220000011000002200000110000022200001111000
result

|2|2|0|0|0|0|0|
|1|1|0|0|0|0|0|
|2|2|0|0|0|0|0|
|1|1|0|0|0|0|0|
|2|2|2|0|0|0|0|
|1|1|1|1|0|0|0|
220000011000002200000110000022200001111000
result of game 9 in thread 1: 1
|2|2|0|0|0|0|0|
|1|1|0|0|0|0|0|
|2|2|0|0|0|0|0|
|1|1|0|0|0|0|0|
|2|2|2|0|0|0|0|
|1|1|1|1|0|0|0|
220000011000002200000110000022200001111000
result of game 9 in thread 3: 1
|2|2|0|0|0|0|0|
|1|1|0|0|0|0|0|
|2|2|0|0|0|0|0|
|1|1|0|0|0|0|0|
|2|2|2|0|0|0|0|
|1|1|1|1|0|0|0|
220000011000002200000110000022200001111000
result of game 9 in thread 2: 1
[4, 6, 6, 8]
winrate of the 0th iteration: 0.6


thread 1 ended, result: 1
thread 0 ended, result: 1
thread 3 ended, result: 1
thread 2 ended, result: 1
(15, 7)
(15, 7)
(15, 7)
(60, 7)
generating data for iteration 1 and game 0


KeyboardInterrupt: 

# Create an Agent

To create the submission, an agent function should be fully encapsulated (no external dependencies).  

When your agent is being evaluated against others, it will not have access to the Kaggle docker image.  Only the following can be imported: Python Standard Library Modules, gym, numpy, scipy, pytorch (1.3.1, cpu only), and more may be added later.



In [ ]:
import functools
# This agent random chooses a non-empty column.

def my_agent(observation, configuration):
    board = functools.reduce(lambda x, y: str(x) + str(y), observation.board)
    game = Connectx(configuration.inarow, configuration.rows, configuration.columns)
    mcts = MCTS(game, nn)
    turn = str(observation.mark)
    
#     print(turn, configuration.rows, configuration.columns)
#     print(board)
    game.print_board(board)
    move = mcts.best_move(board, turn, n_iters=40)
    for a in mcts.game.get_open_cols(board):
        c_aux = mcts.game.make_move(board, a, turn)
        print(f'N: {mcts.nodes_parameters[c_aux][0]} , V: {mcts.nodes_parameters[c_aux][1]}')
#     print(move)
    return move

# Test your Agent

In [ ]:
env.reset()
# Play as the first agent against default "random" agent.
env.run(["negamax", my_agent])
env.render(mode="ipython", width=500, height=450)

# Debug/Train your Agent

In [ ]:
# Play as first position against random agent.
trainer = env.train([None, "random"])

observation = trainer.reset()

while not env.done:
    my_action = my_agent(observation, env.configuration)
    print("My Action", my_action)
    observation, reward, done, info = trainer.step(my_action)
    # env.render(mode="ipython", width=100, height=90, header=False, controls=False)
env.render()

# Evaluate your Agent

In [ ]:
def mean_reward(rewards):
    return sum(r[0] for r in rewards) / float(len(rewards))

# Run multiple episodes to estimate its performance.
print("My Agent vs Random Agent:", mean_reward(evaluate("connectx", [my_agent, "random"], num_episodes=10)))
print("My Agent vs Negamax Agent:", mean_reward(evaluate("connectx", [my_agent, "negamax"], num_episodes=10)))

# Play your Agent
Click on any column to place a checker there ("manually select action").

In [ ]:
# "None" represents which agent you'll manually play as (first or second player).
env.play([my_agent, None], width=500, height=450)

# Write Submission File



In [ ]:
import inspect
import os

def write_agent_to_file(function, file):
    with open(file, "a" if os.path.exists(file) else "w") as f:
        f.write(inspect.getsource(function))
        print(function, "written to", file)

write_agent_to_file(my_agent, "submission.py")

# Validate Submission
Play your submission against itself.  This is the first episode the competition will run to weed out erroneous agents.

Why validate? This roughly verifies that your submission is fully encapsulated and can be run remotely.

In [ ]:
# Note: Stdout replacement is a temporary workaround.
import sys
out = sys.stdout
submission = utils.read_file("/kaggle/working/submission.py")
agent = utils.get_last_callable(submission)
sys.stdout = out

env = make("connectx", debug=True)
env.run([agent, agent])
print("Success!" if env.state[0].status == env.state[1].status == "DONE" else "Failed...")

# Submit to Competition

1. Commit this kernel.
2. View the commited version.
3. Go to "Data" section and find submission.py file.
4. Click "Submit to Competition"
5. Go to [My Submissions](https://kaggle.com/c/connectx/submissions) to view your score and episodes being played.